In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv(r"E:\MIT-6\ML\ML Algos Lab\Data\PlayTennis.csv")
data.head()

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [2]:
#calculate entropy for entire data set ie) the infoD
def calculateINFOD(train_data,targetlabel,classes):
    total_row = train_data.shape[0]
    tot_entr=0
    for c in classes:
        tot_classes = train_data[train_data[targetlabel]==c].shape[0]
        tot_class_entr = -(tot_classes/total_row)*np.log2(tot_classes/total_row)
        tot_entr += tot_class_entr
    return tot_entr

In [3]:
#calculate the entropy for filtered data set 
def calcINFO_of_feature(feature_value,targetlabel,classes):
    no_of_classes = feature_value.shape[0]
    entropy = 0
    
    for c in classes:
        targetlabel_classcount = feature_value[feature_value[targetlabel]==c].shape[0]
        entropy_class = 0
        if targetlabel_classcount != 0:
            prob_class = targetlabel_classcount/no_of_classes
            entropy_class -=  prob_class * np.log2(prob_class)
        entropy += entropy_class
    return entropy
print(calcINFO_of_feature(data[data['Outlook'] == 'Sunny'],'Play Tennis',['yes','no']))

0


In [4]:
#Calculate the gain for each feature 
def calINFORGAIN(feature_name,train_data,targetlabel,classes):
    feature_list = train_data[feature_name].unique() #pass the unqiue values for each feature from all the rows of it
    tot_rows = train_data.shape[0]
    feature_info = 0.0
    
    for val in feature_list:
        feature_valdata = train_data[train_data[feature_name]==val]
        feature_valcount = feature_valdata.shape[0]
        feature_valentropy = calcINFO_of_feature(feature_valdata,targetlabel,classes)
        feature_valprob = feature_valcount/tot_rows
        feature_info+=feature_valprob*feature_valentropy
    return calculateINFOD(train_data,targetlabel,classes)-feature_info
print(calINFORGAIN('Outlook',data,'Play Tennis',['yes','no']))

nan


C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in log2
  import sys
C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys


In [5]:
#find the feature with highest gain among the given features
def calMaxGain(train_data,targetlabel,classes):
    features_list = train_data.columns.drop(targetlabel)
    max_gain = -1
    max_gain_feature = None
    for f in features_list:
        feature_info_gain = calINFORGAIN(f,train_data,targetlabel,classes)
        if max_gain  < feature_info_gain:
            max_gain = feature_info_gain
            max_gain_feature = f
    return max_gain_feature

In [6]:
#generate a sub tree by adding nodes
def generate_subtree(feature,train_data,targetlabel,classes):
    feature_valcount_dict = train_data[feature].value_counts(sort=False)#a dictionary of the count of unqiue features values
    tree = {} #node
    for f_val,count in feature_valcount_dict.iteritems():
        feature_valdata = train_data[train_data[feature]==f_val]
        
        assigned = False
        for c in classes:
            class_count = feature_valdata[feature_valdata[targetlabel]==c].shape[0]
            
            if class_count == count:
                tree[f_val]=c
                train_data= train_data[train_data[feature]!=f_val]
                assigned = True
        if not assigned:
            tree[f_val]="?"
        
        return tree,train_data          

In [7]:
def make_tree(root, prev_feature_value, train_data, targetlabel, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature = calMaxGain(train_data, targetlabel, class_list) #most informative feature
        tree, train_data = generate_subtree(max_info_feature, train_data, targetlabel, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, targetlabel, class_list) #recursive call with updated dataset


In [8]:
def id3(train_data_m, targetlabel):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[targetlabel].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data, targetlabel, class_list) #start calling recursion
    return tree

In [10]:
tree = id3(data,'Play Tennis')
print(tree)

{'Outlook': {'Rain': {'Wind': {'Weak': 'Yes'}}}}
